In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import string
from sklearn.model_selection import train_test_split 
from keras import regularizers


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Embedding
from tensorflow.keras.layers import SpatialDropout1D, Conv1D, Bidirectional, LSTM, Dense, Input, Dropout, GlobalMaxPooling1D
from keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
LABR = pd.read_csv(r'/content/gdrive/MyDrive/Final_Dataset.tsv',delimiter='\t')

In [ ]:
data = LABR

"""**printing the first 3 rows of the data**"""

data.head(3)

,Label,Text
0,4,من أمتع ما قرأت من روايات بلا شك. وحول الشك تد...
1,4,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
2,1,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...


In [ ]:
print("Data contient {} lignes et {} colonnes.".format(data.shape[0], data.shape[1]))

Data contient 63256 lignes et 2 colonnes.


In [ ]:
data.isnull().sum()

Label    0
Text     0
dtype: int64

In [ ]:
print("On a  {} doublons dans Data.".format(data.duplicated().sum()))

data.drop_duplicates(inplace = True)

print("On a  {} doublons dans Data.".format(data.duplicated().sum()))

On a  3030 doublons dans Data.
On a  0 doublons dans Data.


In [ ]:
positive_reviews = data[data["Label"] > 3]
positive_reviews["sentiment"] = 1

negative_reviews = data[data["Label"] < 3]
negative_reviews["sentiment"] = 0

data = pd.concat([positive_reviews, negative_reviews], ignore_index = True)


<ipython-input-9-7a9fa9191a23>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_reviews["sentiment"] = 1
<ipython-input-9-7a9fa9191a23>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews["sentiment"] = 0


In [ ]:
def compte_mots(phrase):
    return len(str(phrase).split())


In [ ]:
data["len_review"] = data["Text"].apply(compte_mots)
positive_reviews['len_review'] = positive_reviews["Text"].apply(compte_mots)
negative_reviews['len_review'] = negative_reviews["Text"].apply(compte_mots)


<ipython-input-11-420b95cd919b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_reviews['len_review'] = positive_reviews["Text"].apply(compte_mots)
<ipython-input-11-420b95cd919b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_reviews['len_review'] = negative_reviews["Text"].apply(compte_mots)


In [ ]:
print("Le maximum de mots utilisé dans les reviews positives est :", max(positive_reviews.len_review))
print("Le moyen de mots utilisé dans les reviews positives est :", np.mean(positive_reviews.len_review))
print("-------------------------------------------------------------------------------------------------------")
print("Le maximum de mots utilisé dans les reviews négatives est :", max(negative_reviews.len_review))
print("Le moyen de mots utilisé dans les reviews négatives est :", np.mean(negative_reviews.len_review))


Le maximum de mots utilisé dans les reviews positives est : 3419
Le moyen de mots utilisé dans les reviews positives est : 63.92189538461538
-------------------------------------------------------------------------------------------------------
Le maximum de mots utilisé dans les reviews négatives est : 1535
Le moyen de mots utilisé dans les reviews négatives est : 60.65378768685077


In [ ]:

data.head(3)
df=data



In [ ]:
def preprocessing(x):
    x = re.sub('@[^\s]+', ' ', x)
    x = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',x)

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030""]+", flags=re.UNICODE)
    emoji_pattern.sub(r'', x)

    ar_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ#'''
    en_punctuations = string.punctuation
    punctuations = ar_punctuations + en_punctuations
    x = x.translate(str.maketrans('', '', punctuations))

    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    x = re.sub(arabic_diacritics, '', str(x)) 

#     x = re.sub("[إأآا]", "ا", x)
#     x = re.sub("ى", "ي", x)
#     x = re.sub("ة", "ه", x)
#     x = re.sub("گ", "ك", x)
#     x = re.sub(r'(.)\1+', r'\1', x)

    return x


In [ ]:
data["Clean_reviews"] = data.Text.apply(lambda x: preprocessing(str(x)))

In [ ]:
print('- Avant le prétraitement \n\n',data["Text"][4])
print("\n-----------------------------------------------\n")
print('- Après le prétraitement \n\n',data["Clean_reviews"][4])

- Avant le prétraitement 

  هذا هو الأدب . هذه هي الرواية !

-----------------------------------------------

- Après le prétraitement 

  هذا هو الأدب  هذه هي الرواية 


In [ ]:
X = data.Clean_reviews
y = data.sentiment

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state = 42)

"""**printing the number of the train set and the test set**"""

print('Train set', X_train.shape)
print('Test set', X_test.shape)


Train set (38815,)
Test set (9704,)


In [ ]:
MAX_VOCAB_SIZE=250000
tokenizer=Tokenizer(num_words=MAX_VOCAB_SIZE,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(X_train)


In [ ]:
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
MAX_SEQUENCE_LENGTH = 3456
# 
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train),
                         maxlen = MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test),
                        maxlen = MAX_SEQUENCE_LENGTH)
# 

In [ ]:
import gensim
Embedding_dim=300
file='/content/gdrive/MyDrive/tweet_cbow_300/tweets_cbow_300'
!iconv -f ISO-8859-1 -t UTF-8 /content/gdrive/MyDrive/tweet_cbow_300/tweets_cbow_300> /content/gdrive/MyDrive/tweets_utf8.txt
file_utf8 = '/content/gdrive/MyDrive/tweets_utf8.txt'
# Initialize an empty embeddings index dictionary
EMBEDDINGS_MATRIX = np.zeros ((vocab_size+1, Embedding_dim)) #EMBEDDINGS MATRIX-11
model = gensim.models.Word2Vec.load(file)
for word, i in word_index.items():
  embedding_vector = None
  if word != "<OOV>":
    if word in model.wv:
      embedding_vector = model.wv[word]
EMBEDDINGS_MATRIX[i] = embedding_vector

In [ ]:
EMBEDDINGS_MATRIX = np.nan_to_num(EMBEDDINGS_MATRIX)

In [ ]:
model = Sequential()
embedding_layer = Embedding(vocab_size +1 , 
                            300, 
                            weights = [EMBEDDINGS_MATRIX], 
                            input_length = MAX_SEQUENCE_LENGTH, 
                            trainable=False)
model.add(embedding_layer)
model.add(Conv1D(filters=300, kernel_size=2, activation='relu'))
model.add(Bidirectional(LSTM(128, dropout=0.2, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
model.compile(optimizer = Adam(learning_rate=0.0001), 
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
r = model.fit(
  X_train,
  y_train,
  batch_size = 64,
  epochs=20,
  validation_data=(X_test, y_test),
  callbacks=[es]
)

Epoch 1/20
607/607 [==============================] - 335s 526ms/step - loss: 0.4664 - accuracy: 0.8360 - val_loss: 0.4447 - val_accuracy: 0.8370
Epoch 2/20
607/607 [==============================] - 315s 519ms/step - loss: 0.4455 - accuracy: 0.8374 - val_loss: 0.4449 - val_accuracy: 0.8370
Epoch 3/20
607/607 [==============================] - 331s 545ms/step - loss: 0.4449 - accuracy: 0.8374 - val_loss: 0.4447 - val_accuracy: 0.8370
Epoch 4/20
607/607 [==============================] - 330s 543ms/step - loss: 0.4448 - accuracy: 0.8374 - val_loss: 0.4447 - val_accuracy: 0.8370
Epoch 5/20
607/607 [==============================] - 330s 544ms/step - loss: 0.4450 - accuracy: 0.8374 - val_loss: 0.4447 - val_accuracy: 0.8370
Epoch 6/20
607/607 [==============================] - 315s 519ms/step - loss: 0.4453 - accuracy: 0.8374 - val_loss: 0.4449 - val_accuracy: 0.8370
Epoch 7/20
607/607 [==============================] - 317s 522ms/step - loss: 0.4451 - accuracy: 0.8374 - val_loss: 0.4450 -